## Hyperparameter Tuning with Keras Tuner

Hyperparameter tuning with [Keras Tuner](https://keras-team.github.io/keras-tuner/) library using an air quality dataset. The dataset is provided by Krish Naik in his github repository. The study is performed to identify the optimum number of hidden layers abd neurons to achieve least error of the model.

In [1]:
# Import libraries
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

To work with Keras Tuner, tensorflow version should be 2.0 or higher. Therefore let's check the tensorflow version installed in the current environment.

In [2]:
# Tensorflow version
import tensorflow as tf
tf.__version__

'2.3.1'

In [3]:
# Import data set
df=pd.read_csv('Real_Combine.csv')

In [4]:
df.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


The PM 2.5 (Fine Particulate Matter) feature is the dependent feature of the study.Let's divide the data into independent and dependent features.

In [5]:
# PM 2.5 is the dependent feature and the others are independent features
X=df.iloc[:,:-1] # Independent feature
y=df.iloc[:,-1] # Dependent feature

## Hyperparameters
1. How many numbers of hidden layers we should have?
2. How many number of neurons we should have in hidden layers?
3. Learning rate

The function below defines a neural network with multiple values for each parameter of the neural network. Performing a random search among the combinations of the parameters will lead to find the combination of parameters with the least error.

The loss function use is mean absolute error, because this is a regression problem.

In [6]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)): # Use 2 to 20 hidden layers
        model.add(layers.Dense(units= hp.Int('units_'+ str(i),
                                            min_value= 32,
                                            max_value= 512,
                                            step= 32), # Number of neurons in each layer can vary from 32 to 512
                              activation= 'relu')) # Use relu activation function in each hidden layer
        model.add(layers.Dense(1, activation= 'linear')) # Output layer has only one node (beacause this is a regression problem)
        model.compile(
            optimizer= keras.optimizers.Adam(
                hp.Choice('learning_rate', [1e-2,1e-3,1e-4])), # Select one value from the list
        loss= 'mean_absolute_error',
        metrics=['mean_absolute_error'])
        return model

Now we have compiled a model for the study. Now we can create a tuner object with the Keras Tuner (kerastuner.tuners).

In [7]:
tuner = RandomSearch(
    build_model, 
    objective= 'val_mean_absolute_error', 
    max_trials=5, # Number of trials each model executed
    executions_per_trial=3, # Number of models should be executed
    directory= 'project1', # All the weights are recorded in this directory
    project_name= 'Air Quality Index')

INFO:tensorflow:Reloading Oracle from existing project project1\Air Quality Index\oracle.json
INFO:tensorflow:Reloading Tuner from project1\Air Quality Index\tuner0.json


In [8]:
# Summary of the random search
tuner.search_space_summary()

Search space summary
Default search space size: 3
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [9]:
# Split data into train and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0 )

In [10]:
 # Train and test the models with 5 epochs
tuner.search(X_train, y_train, epochs= 5, validation_data=(X_test, y_test))

INFO:tensorflow:Oracle triggered exit


In [11]:
# Summary of the results
tuner.results_summary()

Results summary
Results in project1\Air Quality Index
Showing 10 best trials
Objective(name='val_mean_absolute_error', direction='min')
Trial summary
Hyperparameters:
num_layers: 3
units_0: 160
learning_rate: 0.01
Score: 50.7225227355957
Trial summary
Hyperparameters:
num_layers: 19
units_0: 32
learning_rate: 0.001
Score: 61.6109873453776
Trial summary
Hyperparameters:
num_layers: 16
units_0: 352
learning_rate: 0.0001
Score: 63.73135121663412
Trial summary
Hyperparameters:
num_layers: 3
units_0: 32
learning_rate: 0.001
Score: 64.04669825236003
Trial summary
Hyperparameters:
num_layers: 15
units_0: 64
learning_rate: 0.0001
Score: 75.27729415893555


The best score with only the 5 epochs is found as 75.277 and the parameters of that model are as below. 
* num_layers: 15
* units_0: 64
* learning_rate: 0.0001